In [0]:
import pyspark.sql.functions as sf

In [0]:
# Registering Empty Dataframe with up to 5 rows
schema = ("ID integer")
data = list(range(5))

df = spark.createDataFrame(data=data,schema=schema)

**Working With NULLs**

In [0]:
nulls_df = spark.range(1)
nulls_df = nulls_df.select(sf.lit(None).alias("NULL_Column")).union(nulls_df.select(sf.lit(1)))

nulls_df = nulls_df.select(
    "NULL_Column",
    sf.nvl2("NULL_Column", sf.lit("Value-in"), sf.lit(None)).alias("nvl2"),
    sf.ifnull(sf.col("NULL_Column").cast("string"), sf.lit("Null Value")).alias("ifnull"),
    sf.coalesce(sf.col("NULL_Column").cast("string"),"nvl2","ifnull").alias("coalesce")
    )
nulls_df.display()

# by default is "any"
nulls_df.dropna("any").display()
# if all then all columns needs to have NULL value to be row deleted
nulls_df.dropna("all").display()
# if subset is provided then by default "any" Null vlaue in a given colum will remove row
nulls_df.dropna(subset=["NULL_Column","nvl2","ifnull"]).display()
# if subset is provided with "all" then all provided columns needs to be NULL value to remove row
nulls_df.dropna("all",subset=["NULL_Column","nvl2","ifnull"]).display()

In [0]:
nulls_df.display()
# this will fill NULL values in columns of string
nulls_df.fillna("null Values filled in").display()
# this will fill NULL values in columns of integer
nulls_df.fillna(0).display()
# this will fill NULLs in 2 specific column of different Data type
nulls_df.fillna({"NULL_Column":0, "nvl2":"Null value"}).display()

**Date & TimeStamps**

In [0]:
# easy way to register Dataframe
date_df = spark.range(5)

# Date and Time fucntions
date_df.select(
    sf.current_date().alias("Current_Date"),
    sf.current_timestamp().alias("Current_DateTime"),
    sf.date_add("Current_Date",5).alias("5 Days in future"),
    sf.date_sub("Current_Date",5).alias("5 Days in past"),
    sf.lit("2025-01-01").alias("StartDate_string"),
    sf.to_date(sf.lit("2025-01-01"), "yyyy-MM-dd").alias("StartDate_date"),
    sf.to_timestamp(sf.lit("2025-01-01"), "yyyy-MM-dd").alias("StartDate_Timestamp"),
    sf.date_diff("StartDate_date","Current_Date").alias("Date Difference"),
    sf.months_between("StartDate_date","Current_Date").alias("Months Difference")
).limit(2).display()

**Number Functions**

In [0]:
display(df.select(
    sf.lit(5).alias("Number"),
    sf.pow("Number",3).alias("Power to 3"),
    (sf.col("Number") * 1.5).alias("Multiplication"),
    sf.round("Multiplication", 0).alias("Rounded Multiplication"),
    sf.floor("Multiplication").alias("Rounded Down Multiplication")
                ).limit(1)
        )

**String Functions**

In [0]:
display(df.select(
    sf.lit("     Hello world     ").alias("String_Value"),
    sf.ltrim("String_Value").alias("Left Trim"),
    sf.rtrim("String_Value").alias("Right Trim"),
    sf.trim("String_Value").alias("Right Trim"),
    sf.upper("String_Value").alias("Upper Case"),
    sf.lower("String_Value").alias("Lower Case"),
    sf.initcap("String_Value").alias("Word Capitalized"),
    sf.lpad(sf.lit("123"),5,0).alias("lpad"), 
    sf.rpad(sf.lit("123"),5,0).alias("rpad")
                ).limit(2)
        )

**Regular Expressions**

In [0]:
df_retail = (spark.read.format("csv")
                       .options(header=True)
                       .load("/Volumes/definitive_spark/default/data/data/retail-data/by-day/2010-12-01.csv")
            )
display (df_retail.limit(5))

In [0]:
df_retail.select(
    "Description",
    sf.regexp_replace(
        sf.regexp_replace(
            sf.regexp_replace("Description", "WHITE|RED", "Color"),  # processed first
                                         "HEARTS", "Thing"), # processed second
                                         "HEART",'NO' # processed last
                                                    ).alias("Regex Replace"),
    sf.translate("Description", "WHITE", "51243").alias("Reges letter replace"),
    sf.regexp_extract("Description", "(WHITE|RED)|(COAT)", 0).alias("Regex Extract")
                                        # 0 - returns first match
                                        # 1 - return first match from first group (WHITE or RED)
                                        # 2 - return first match from second group (COAT)
).limit(5).display()